# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Apathy
df = pd.read_csv('Apathy_with_MethylomeDetails.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Apathy
(458, 3572)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,MDS_Apathy_Category,cg19958696,cg05962325,...,cg19795267,cg16922840,cg19628277,cg26480988,cg01673347,cg12994818,cg09008531,cg16463148,cg04021430,cg04863888
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.010314,0.057597,...,0.751058,0.901978,0.580283,0.548182,0.725312,0.092118,0.723395,0.129795,0.017529,0.026699
1,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.014601,0.068728,...,0.630319,0.851560,0.521620,0.495925,0.830013,0.076462,0.728590,0.246918,0.039718,0.024071
2,200991620021_R03C01,3002,2,1,2,67.6,4,0,0.014465,0.043749,...,0.756439,0.861054,0.530380,0.500924,0.792804,0.074791,0.660346,0.263525,0.029293,0.021847
3,200989060105_R03C01,3003,2,1,2,56.7,3,0,0.011637,0.060551,...,0.739150,0.858457,0.566978,0.603712,0.660141,0.103559,0.718634,0.143732,0.025689,0.023316
4,200989060135_R07C01,3004,1,2,0,59.4,3,0,0.013334,0.038642,...,0.655485,0.867540,0.546537,0.511772,0.802469,0.033789,0.634950,0.193630,0.023717,0.023474


In [4]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'MDS_Apathy_Category','Age','AgeCate'], axis=1)
print(df1.shape)
df1.head(2)

(458, 3565)


,APPRDX,cg19958696,cg05962325,cg15093662,cg04873221,cg05903298,cg19020436,cg14994056,cg05973813,cg18337422,...,cg19795267,cg16922840,cg19628277,cg26480988,cg01673347,cg12994818,cg09008531,cg16463148,cg04021430,cg04863888
0,2,0.010314,0.057597,0.020549,0.015659,0.011741,0.014318,0.035177,0.014771,0.013507,...,0.751058,0.901978,0.580283,0.548182,0.725312,0.092118,0.723395,0.129795,0.017529,0.026699
1,1,0.014601,0.068728,0.019329,0.021747,0.012923,0.017399,0.033939,0.020282,0.017160,...,0.630319,0.851560,0.521620,0.495925,0.830013,0.076462,0.728590,0.246918,0.039718,0.024071


## RanFor 100 times, CpG only 

In [6]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [7]:
dframe1.to_csv ("Apathy_AllCpG_Accuracy_100RF.csv", index=False)

In [8]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.490,0.086023
1,0.515,0.059372


## RanFor 1000 times. Samples = 50

In [9]:
df2 = df1.drop (['APPRDX'], axis=1)

In [11]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [12]:
print(dframe1.head(3))
dframe1.to_csv ("Apathy_Rand50CpG_100RF.csv")

   Accuracy   Std_Dev
0     0.475  0.147054
1     0.510  0.091652
2     0.410  0.115758


## RanFor 1000 times. Samples = 10

In [13]:
np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(10, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [14]:
print(dframe1.head(3))
dframe1.to_csv ("Apathy_Rand10CpG_100RF.csv")

   Accuracy   Std_Dev
0     0.475  0.147054
1     0.510  0.091652
2     0.410  0.115758
